In [1]:
# Jiaqi Xue 

# Test model and show result

import os

#if use GPU
'''
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"  #选择哪一块gpu
config = ConfigProto()
config.allow_soft_placement=True #如果你指定的设备不存在，允许TF自动分配设备
config.gpu_options.per_process_gpu_memory_fraction=0.7  #分配百分之七十的显存给程序使用，避免内存溢出，可以自己调整
config.gpu_options.allow_growth = True   #按需分配显存，这个比较重要
session = InteractiveSession(config=config)
'''
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf  
tf.compat.v1.disable_eager_execution() 
import keras.backend as K
from tensorflow.keras.optimizers import RMSprop,Adam
K.set_learning_phase(0) #0test
from keras.models import load_model
from sklearn.metrics import r2_score
from utilities_multi import *



#R-squared function
def r2(y_true, y_pred):
    a = K.square(y_pred - y_true)
    b = K.sum(a)
    c = K.mean(y_true)
    d = K.square(y_true - c)
    e = K.sum(d)
    f = 1 - b/e
    return f
#test function
def test(MODEL_PATH):
    
    print("MODEL_PATH is",MODEL_PATH)
    print("loading model and reading data..................")
    model = load_model(MODEL_PATH,custom_objects={'r2': r2},compile=False)
    losses = {
            'outputs_r1':'MSE',
            'outputs_r2':'MSE',
            'outputs_c':'sparse_categorical_crossentropy',

        }
    my_loss_weights = {
    'outputs_r1':1,
    'outputs_r2':1,    
    'outputs_c':100,
    }
    me = {
            'outputs_r1':r2,
            'outputs_r2':r2,            
            'outputs_c':'sparse_categorical_accuracy',

        }
    model.compile(loss = losses, loss_weights=  my_loss_weights, optimizer=Adam(),metrics=me)

    graph = tf.get_default_graph()

    #testing set 
    X_test41to50, y_test41to50_r1,y_test41to50_r2,y_test41to50_c = read_data(data_path='./Data/data_csv_41to49_pro.txt', split="train") # train
    print("test sample amount for this subject is",len(y_test41to50_r1))
    print('testing..................')

    test = model.evaluate(X_test41to50,[y_test41to50_r1,y_test41to50_r2,y_test41to50_c])
    print('Accuracy of motion state classification is',test[-1])

    pre_result_r1,pre_result_r2,pre_result_c = model.predict(X_test41to50)
    

    pred_acc1 = r2_score(y_test41to50_r1, pre_result_r1,multioutput="raw_values")
    pred_acc2 = r2_score(y_test41to50_r2, pre_result_r2,multioutput="raw_values")

    print('r2 of right knee angle prediction is',pred_acc1[0])

    print('r2 of left knee angle prediction is',pred_acc2[0])


    time=a=np.arange(0,len(pre_result_r1)*0.02,0.02)

    pre_result_c=np.array(pre_result_c)
    pre_result_c=np.argmax(pre_result_c,axis=1)
    pre_result_c=pre_result_c.reshape(len(pre_result_c),1)

    
    print("showing part of testing result..................")
    print("result figures are saved in ./Result_figures..................")
    plt.style.use("ggplot")  
    plt.figure(figsize=(10, 7))  
    plt.title('Right Knee Angle Prediction')
    plt.xlabel('Time/s')
    plt.ylabel('Angle/°') 
    plt.plot(time[0:2200], y_test41to50_r1[0:2200,0], linestyle='-', color='k', label='True')  
    plt.plot(time[0:2200], pre_result_r1[0:2200,0], linestyle=':', color='m',label='Predicted') 
    plt.legend(loc=0)  
    plt.savefig('./Result_figures/Right_knee.jpg')
    plt.show()

    plt.style.use("ggplot") 
    plt.figure(figsize=(10, 7))  
    plt.title('Left Knee Angle Prediction')
    plt.xlabel('Time/s')
    plt.ylabel('Angle/°') 
    plt.plot(time[0:2200], y_test41to50_r2[0:2200,0], linestyle='-', color='k', label='True') 
    plt.plot(time[0:2200], pre_result_r2[0:2200,0], linestyle=':', color='m',label='Predicted')
    plt.legend(loc=0) 
    plt.savefig('./Result_figures/Left_knee.jpg')
    plt.show()
    
    
    plt.style.use("ggplot")  
    plt.figure(figsize=(10, 7))  
    plt.title('Motion State Classification')
    plt.xlabel('Time/s')
    plt.ylabel('State') 
    plt.plot(time[0:2200], y_test41to50_c[0:2200,0], linestyle='-', color='k', label='True')  
    plt.plot(time[0:2200], pre_result_c[0:2200,0], linestyle=':', color='m',label='Predicted')  
    plt.legend(loc=0)  
    plt.savefig('./Result_figures/Motion_state.jpg')
    plt.show()    
    

    
#Testing    
MODEL_PATH = r'./Model/model.h5'


test(MODEL_PATH)





D:\Anaconda3\envs\tensorflow-keras-gpu\lib\site-packages\keras\backend.py:401: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


MODEL_PATH is ./Model/model.h5
loading model and reading data..................
len(labels_r1) 10918
(labels_r1[0]) [95.62129166]
(labels_c[0]) [0.]
test sample amount for this subject is 10918
testing..................


D:\Anaconda3\envs\tensorflow-keras-gpu\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy of motion state classification is 0.9312145
r2 of right knee angle prediction is 0.9793026229262347
r2 of left knee angle prediction is 0.9752119161533099
showing part of testing result..................
result figures are saved in ./Result_figures..................
